In [2]:
import pandas as pd
import numpy as np
import json
import os

In [12]:
drugs = set()

In [13]:
indir = "/scratch/jhm3ab/antibiotic_bii/prescriptions_data/"
for yeardir in os.listdir(indir):
    d = os.path.join(indir, yeardir)
    for filename in os.listdir(d):
        fin = os.path.join(d, filename)
        f = open(fin, 'r')
        new = json.load(f)
        drugs.update(new.keys())
        f.close()

In [7]:
len(drugs)

104212

In [14]:
f = open('alldrugsndc.txt', 'w')
for ndc in drugs:
    f.write(ndc + '\n')

In [12]:
ndcfile = "/scratch/jhm3ab/antibiotic_bii/ndclist/product.txt"
ndcdata = pd.read_csv(ndcfile, sep = '\t', encoding='unicode_escape')

In [7]:
classes = ['Aminoglycoside', 
           'Penicillin',
 'Polymyxin',
 'Lipopeptide',
 'Glycopeptide',
 'Carbapenem',
 'Cephalosporin',
 'Monobactam',
 'Trimethoprim',
 'Chloramphenicol',
 'Macrolide',
 'Oxazolidinone',
 'Quinolone',
 'Tetracycline',
 'Glycylcycline']

In [17]:
len(classes)

15

In [13]:
pharm=ndcdata['PHARM_CLASSES'].unique().tolist()

In [19]:
search = []
failed = []
for c in classes:
    found = False
    for p in pharm:
        p = str(p)
        if c.lower() in p.lower():
            search.append(p)
            found = True
    if not found:
        failed.append(c)

In [20]:
failed

['Trimethoprim', 'Chloramphenicol', 'Glycylcycline']

In [21]:
filtered = ndcdata.loc[ndcdata['PHARM_CLASSES'].isin(search)]
ndcfilter = filtered['PRODUCTNDC'].unique().tolist()
ndc = [n.replace('-','') for n in ndcfilter]

In [22]:
found = []
for key in drugs:
    for n in ndc:
        if n in key:
            found.append(key)

In [23]:
len(found)

1175

In [27]:
f = open('all_antibiotics.txt', 'w')
for ndc in found:
    f.write(ndc + '\n')

In [4]:
f = open('all_antibiotics.txt', 'r')
found = f.read().splitlines()

In [5]:
all_antibiotics = {}

In [8]:
indir = "/scratch/jhm3ab/antibiotic_bii/prescriptions_data/"
time = [2016,1]
for yeardir in os.listdir(indir):
    d = os.path.join(indir, yeardir)
    time[1] = 1
    for filename in os.listdir(d):
        fin = os.path.join(d, filename)
        f = open(fin, 'r')
        new = json.load(f)
        data = {}
        for key in new:
            if key in found:
                data[key] = new[key]
        f.close()
        all_antibiotics[tuple(time)] = data
        time[1] += 1
    time[0] += 1

In [9]:
out = {}
for key in all_antibiotics.keys():
    out[''.join([str(value) for value in key])] = all_antibiotics[key]
with open('filtered_antibiotics/all_antibiotics.json', "w") as f:
    json.dump(out, f)

In [14]:
for c in classes:
    search = []
    for p in pharm:
        p = str(p)
        if c.lower() in p.lower():
            search.append(p)
    filtered = ndcdata.loc[ndcdata['PHARM_CLASSES'].isin(search)]
    ndcfilter = filtered['PRODUCTNDC'].unique().tolist()
    ndc = [n.replace('-','') for n in ndcfilter]
    with open('antibiotic_class_ndc/' + c + '.txt', "w") as f:
        for key in found:
            for n in ndc:
                if n in key:
                    f.write(key + '\n')